<a href="https://colab.research.google.com/github/onehungrybird/Agentic_AI_travel_planner/blob/main/Agentic_AI_travel_planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain googlemaps amadeus mistralai langchain-community mistralai ollama torch
!pip install transformers accelerate bitsandbytes --quiet
!pip install -U bitsandbytes

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

In [3]:
import bitsandbytes
print("bitsandbytes version:", bitsandbytes.__version__)  # Should show 0.41.1

bitsandbytes version: 0.45.4


In [4]:
import os, torch
import googlemaps
from amadeus import Client, ResponseError
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Configure 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
mistral_llm = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [49]:
GOOGLE_MAPS_API_KEY = userdata.get('G_MAPS')
AMADEUS_API_KEY = userdata.get('AMADEUS_API_KEY')
AMADEUS_API_SECRET = userdata.get('AMADEUS_API_SECRET')

# Initialize API Clients
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)
amadeus = Client(client_id=AMADEUS_API_KEY, client_secret=AMADEUS_API_SECRET)

In [88]:
def search_flights(origin, destination, departure_date, budget):
    """Fetches real-time flight data from Amadeus API."""
    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=origin,
            destinationLocationCode=destination,
            departureDate=departure_date,
            maxPrice=budget
        )
        return response.data[:5]  # Return top  flights
    except ResponseError as error:
        print(f"Error fetching flights: {error}")
        return "No flights found"

In [89]:
def search_hotels(destination, budget):
    """Fetches best hotel offers from Amadeus API."""
    try:
        response = amadeus.shopping.hotel_offers_search.get(
            cityCode=destination,
            priceRange=f"0-{budget}",
            radius=10
        )
        return response.data[:5]  # Return top  hotels
    except ResponseError as error:
        print(f"Error fetching hotels: {error}")
        return "No hotels found"

In [90]:
def find_attractions(destination):
    """Fetches top attractions using Google Places API."""
    geocode_result = gmaps.geocode(destination)
    if not geocode_result:
        return "Location not found"

    lat, lng = geocode_result[0]["geometry"]["location"].values()

    places = gmaps.places_nearby(
        location=(lat, lng),
        radius=5000,
        type="tourist_attraction"
    )

    return [p["name"] for p in places.get("results", [])[:5]]  # Top 5 attractions

In [95]:
class TravelAgent:
    def __init__(self, origin, destination, budget, departure_date, model, tokenizer, amadeus_client, gmaps_client):
        self.origin = origin
        self.destination = destination
        self.budget = budget
        self.departure_date = departure_date
        self.model = model
        self.tokenizer = tokenizer

        # Use pre-initialized API clients
        self.amadeus = amadeus_client
        self.gmaps = gmaps_client

    def search_flights(self):
        """Search for best flight options using Amadeus API."""
        try:
            response = self.amadeus.shopping.flight_offers_search.get(
                originLocationCode=self.origin,
                destinationLocationCode=self.destination,
                departureDate=self.departure_date,
                adults=1
            )
            return response.data
        except Exception as e:
            print(f"Error fetching flights: {e}")
            return []

    def search_hotels(self):
        """Search for best hotel options using Amadeus API."""
        try:
            response = self.amadeus.reference_data.locations.hotels.by_city.get(
                cityCode=self.destination
            )

            hotels = response.data  # Get hotel locations
            if not hotels:
                return "No hotels found"

            # Extract hotel IDs
            hotel_ids = [hotel["hotelId"] for hotel in hotels[:5]]  # Get top 5 hotels

            # Fetch offers for these hotels
            response_offers = self.amadeus.shopping.hotel_offers_search.get(
                hotelIds=','.join(hotel_ids),
                currency="USD"
            )

            offers = response_offers.data
            return offers if offers else "No hotel offers found"

        except Exception as error:
            print(f"Error fetching hotels: {error}")
            return "No hotels found"


    def create_itinerary(self):
        """Generates a personalized 5-day itinerary using Mistral AI."""
        self.tokenizer.pad_token = self.tokenizer.eos_token

        prompt = f"""
        Create a 5-day travel itinerary from {self.origin} to {self.destination}.
        Include flight details, hotel recommendations, and top attractions.
        """

        input_ids = self.tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda" if torch.cuda.is_available() else "cpu")

        with torch.no_grad():
            output = self.model.generate(input_ids=input_ids, max_length=1000, pad_token_id=self.tokenizer.eos_token_id)

        itinerary = self.tokenizer.decode(output[0], skip_special_tokens=True)
        return itinerary


    def plan_trip(self):
        """Runs the full trip planning process."""
        flights, hotels = self.search_flights(), self.search_hotels()
        itinerary = self.create_itinerary()

        return {
            "destination": self.destination,
            "best_flights": flights,
            "best_hotels": hotels,
            "itinerary": itinerary
        }

In [102]:
# Define user inputs
origin = "BOM"  #mumbai
destination_city = "DEL"  #Delhi
budget = 5000
departure_date = "2025-06-15"

# Initialize TravelAgent
agent = TravelAgent(origin, destination_city, budget, departure_date, mistral_llm, tokenizer, amadeus, gmaps)

# Plan the trip
trip_plan = agent.plan_trip()

# Display Trip Summary Header
print(f"\n🔹 **Trip from {origin} to {destination_city}** 🔹\n")

# Display Best Flights
print("✈️ **Best Flights:**")
best_flights = trip_plan.get("best_flights", [])

if best_flights:
    for flight in best_flights:
        if "itineraries" in flight and flight["itineraries"]:
            itinerary = flight["itineraries"][0]  # First itinerary
            if "segments" in itinerary and itinerary["segments"]:
                segment = itinerary["segments"][0]  # First segment

                carrier_code = segment.get("carrierCode", "Unknown Airline")
                flight_number = segment.get("number", "N/A")
                departure_time = segment.get("departure", {}).get("at", "N/A")
                arrival_time = segment.get("arrival", {}).get("at", "N/A")
                price = flight.get("price", {}).get("total", "N/A")
                duration = segment.get("duration", "N/A")

                print(f" - {carrier_code}{flight_number}: ${price} and duration {duration}")
                print(f"   ⏰ Departure: {departure_time} | Arrival: {arrival_time}")
            else:
                print(" - No segment details found.")
        else:
            print(" - No itinerary details found.")
else:
    print("No flight options found.")

# Display Best Hotels
print("\n🏨 **Best Hotels:**")
best_hotels = trip_plan.get("best_hotels", [])

if isinstance(best_hotels, list) and best_hotels:
    for hotel in best_hotels:
        hotel_name = hotel.get("hotel", {}).get("name", "Unknown Hotel")
        price = hotel.get("offers", [{}])[0].get("price", {}).get("total", "N/A")
        print(f" - {hotel_name} at ${price} total")
else:
    print("No hotel options found.")

# Display Itinerary
print("\n📅 **Itinerary:**")
itinerary = trip_plan.get("itinerary", "No itinerary generated.")
print(itinerary)

Error fetching hotels: [400]
[hotelIds=BWDEL553] Provider Error - INVALID OR MISSING DATA
[hotelIds=ONDELCLA] Provider Error - NO ROOMS AVAILABLE AT REQUESTED PROPERTY
[hotelIds=SCDEL153] Provider Error - RATE NOT AVAILABLE FOR REQUESTED DATES
[hotelIds=TJDEL859,TJDELGUR] Provider Error - INVALID PROPERTY CODE

🔹 **Trip from BOM to DEL** 🔹

✈️ **Best Flights:**
 - AI2424: $62.19 and duration PT2H20M
   ⏰ Departure: 2025-06-15T07:00:00 | Arrival: 2025-06-15T09:20:00
 - AI2994: $67.44 and duration PT2H15M
   ⏰ Departure: 2025-06-15T10:30:00 | Arrival: 2025-06-15T12:45:00
 - AI2928: $72.69 and duration PT2H10M
   ⏰ Departure: 2025-06-15T06:30:00 | Arrival: 2025-06-15T08:40:00
 - AI2960: $72.69 and duration PT2H10M
   ⏰ Departure: 2025-06-15T11:55:00 | Arrival: 2025-06-15T14:05:00
 - AI864: $72.69 and duration PT2H15M
   ⏰ Departure: 2025-06-15T07:00:00 | Arrival: 2025-06-15T09:15:00
 - AI809: $72.69 and duration PT2H15M
   ⏰ Departure: 2025-06-15T10:00:00 | Arrival: 2025-06-15T12:15:00
 -